<a href="https://colab.research.google.com/github/adeepH/CASE-2021-Task-1/blob/main/Notebooks/bert_multi_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
sen=[]
l=[]
with open('/content/drive/MyDrive/en-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    l.append(x['label'])
    sen.append(x['sentence'].strip())

In [ ]:
ssen=[]
sl=[]
with open('/content/drive/MyDrive/es-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    sl.append(x['label'])
    ssen.append(x['sentence'].strip())
psen=[]
pl=[]
with open('/content/drive/MyDrive/pr-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    pl.append(x['label'])
    psen.append(x['sentence'].strip())

In [ ]:
x=sen
x.extend(ssen)
x.extend(psen)
y=l
y.extend(sl)
y.extend(pl)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 28.1MB/s 
     |████████████████████████████████| 3.3MB 42.8MB/s 
     |████████████████████████████████| 901kB 47.8MB/s 


In [ ]:
from transformers import BertTokenizer
from transformers import TFBertModel

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
tokenizer('hi i am pawan kalyan',return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101, 11520,   177, 10392, 10931, 14394, 10730, 47785, 10115,
          102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling1D,Dropout,LSTM
from tensorflow.keras.models import Model
inp1=Input(shape=(128,),dtype='int32')
inp2=Input(shape=(128,),dtype='int32')
emb=model(inp1,attention_mask=inp2)[0]
l=CuDNNLSTM(128,return_sequences=True)(emb)
l=GlobalAveragePooling1D()(l)
l=Dense(50,activation='relu')(l)
l=Dropout(0.2)(l)
l=Dense(1,activation='sigmoid')(l)
m=Model(inputs=[inp1,inp2],outputs=l)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
for i in m.layers[:3]:
  i.trainable=False

In [ ]:
from sklearn.model_selection import train_test_split
x,x1,y,y1=train_test_split(x,y,test_size=0.2)

In [ ]:
d=tokenizer(x,padding=True,truncation=True,return_token_type_ids=False,max_length=128)
d1=tokenizer(x1,padding=True,truncation=True,return_token_type_ids=False,max_length=128)

In [ ]:
import numpy as np
q1=np.array(d['input_ids'])
q2=np.array(d['attention_mask'])
q3=np.array(d1['input_ids'])
q4=np.array(d1['attention_mask'])

In [ ]:
y=np.array(y)
y1=np.array(y1)

In [ ]:
import keras
m.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5),loss='binary_crossentropy',metrics=['accuracy'])
m.fit([q1,q2],y,epochs=3,batch_size=32,validation_data=([q3,q4],y1))

Epoch 1/3
669/669 [==============================] - 267s 383ms/step - loss: 0.3054 - accuracy: 0.8680 - val_loss: 0.3113 - val_accuracy: 0.8621
Epoch 2/3
669/669 [==============================] - 253s 378ms/step - loss: 0.2969 - accuracy: 0.8700 - val_loss: 0.3128 - val_accuracy: 0.8561
Epoch 3/3
669/669 [==============================] - 253s 379ms/step - loss: 0.2922 - accuracy: 0.8687 - val_loss: 0.3088 - val_accuracy: 0.8637


In [ ]:
m.save_weights('/content/drive/MyDrive/bert multi cased model')

In [ ]:
test_model=Model(inputs=[inp1,inp2],outputs=l)

In [ ]:
import keras
test_model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
test_model.load_weights('/content/drive/MyDrive/bert multi cased model')

In [ ]:
pred=test_model.predict([q3,q4])
p=[0 if i[0]<=0.5 else 1 for i in pred]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y1,p,digits=3))

              precision    recall  f1-score   support

           0      0.926     0.921     0.924      4375
           1      0.655     0.671     0.663       975

    accuracy                          0.876      5350
   macro avg      0.790     0.796     0.793      5350
weighted avg      0.877     0.876     0.876      5350

